In [90]:
import pandas as pd
import glob
import simplejson as json
import re
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
#from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import scipy.integrate as integrate
from scipy.sparse import coo_matrix, vstack
import numpy
from scipy.sparse import vstack
from sklearn.metrics.pairwise import cosine_similarity
import nltk

In [2]:
import ijson


In [3]:
pip install sklearn.cross_validation

ERROR: Could not find a version that satisfies the requirement sklearn.cross_validation (from versions: none)
ERROR: No matching distribution found for sklearn.cross_validation
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install ijson

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
#read the json file

In [137]:
def read_file():
    read_files = glob.glob('*.json')
    for f in range(len(read_files)):
        with open(read_files[f]) as infile:
            full_data =json.load(infile)
    return full_data        

In [139]:
full_data=read_file()

In [141]:
#(full_data)

In [142]:
full_data[0]

{'id': 10259,
 'cuisine': 'greek',
 'ingredients': ['romaine lettuce',
  'black olives',
  'grape tomatoes',
  'garlic',
  'pepper',
  'purple onion',
  'seasoning',
  'garbanzo beans',
  'feta cheese crumbles']}

In [9]:
len(full_data)

39774

In [ ]:
#converting the dictionary to a dataframe

In [10]:

idd=[]
cuisine=[]
ingredients=[]
for i in range(len(full_data)):
    idd.append(full_data[i]['id'])
    cuisine.append(full_data[i]['cuisine'])
    ingredients.append(full_data[i]['ingredients'])
    

In [11]:
df = pd.DataFrame({'id':idd, 
                   'cuisine':cuisine, 
                   'ingredients':ingredients})
print(df.head(5))

      id      cuisine                                        ingredients
0  10259        greek  [romaine lettuce, black olives, grape tomatoes...
1  25693  southern_us  [plain flour, ground pepper, salt, tomatoes, g...
2  20130     filipino  [eggs, pepper, salt, mayonaise, cooking oil, g...
3  22213       indian                [water, vegetable oil, wheat, salt]
4  13162       indian  [black pepper, shallots, cornflour, cayenne pe...


In [12]:
df['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [13]:
new = []
for s in df['ingredients']:
    s = ' '.join(s)
    new.append(s)

In [14]:
df['jointedingredients'] = new

In [15]:
print(df.head(10))

      id      cuisine                                        ingredients  \
0  10259        greek  [romaine lettuce, black olives, grape tomatoes...   
1  25693  southern_us  [plain flour, ground pepper, salt, tomatoes, g...   
2  20130     filipino  [eggs, pepper, salt, mayonaise, cooking oil, g...   
3  22213       indian                [water, vegetable oil, wheat, salt]   
4  13162       indian  [black pepper, shallots, cornflour, cayenne pe...   
5   6602     jamaican  [plain flour, sugar, butter, eggs, fresh ginge...   
6  42779      spanish  [olive oil, salt, medium shrimp, pepper, garli...   
7   3735      italian  [sugar, pistachio nuts, white almond bark, flo...   
8  16903      mexican  [olive oil, purple onion, fresh pineapple, por...   
9  12734      italian  [chopped tomatoes, fresh basil, garlic, extra-...   

                                  jointedingredients  
0  romaine lettuce black olives grape tomatoes ga...  
1  plain flour ground pepper salt tomatoes ground... 

In [ ]:
#vectorize the jointed ingredients 

In [16]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['jointedingredients'])

In [17]:
print(X)

  (0, 738)	0.3343204746101372
  (0, 522)	0.14568369866765699
  (0, 958)	0.3040361765035925
  (0, 184)	0.20748802168948122
  (0, 1088)	0.38853112215987895
  (0, 2403)	0.23007896012035983
  (0, 1872)	0.1645493089953018
  (0, 2181)	0.23913220198081458
  (0, 1998)	0.10208411357610164
  (0, 1094)	0.10531073154596084
  (0, 2770)	0.15183517837377775
  (0, 1169)	0.35031170238526027
  (0, 1868)	0.26100924108701357
  (0, 247)	0.1398962004921347
  (0, 1527)	0.2663871237012894
  (0, 2295)	0.3426671291173114
  (1, 1860)	0.11553051231205462
  (1, 2877)	0.2107912072389798
  (1, 1707)	0.20955261084571736
  (1, 1666)	0.3984043568116592
  (1, 676)	0.2182707943085128
  (1, 2983)	0.27471923855187746
  (1, 1190)	0.18772612842727063
  (1, 898)	0.19924115060270703
  (1, 2747)	0.2696867216449215
  :	:
  (39772, 676)	0.10613772616897188
  (39772, 2349)	0.052210298712866955
  (39772, 1013)	0.08838193470073283
  (39772, 1094)	0.06122277701590711
  (39773, 490)	0.2569778430076833
  (39773, 2293)	0.400871723348894

In [ ]:
#label encoding the cusines

In [18]:
le = preprocessing.LabelEncoder()
le.fit(df['cuisine'])
df['cuisine']=le.transform(df['cuisine']) 

In [19]:
df['cuisine'].value_counts()

9     7838
13    6438
16    4320
7     3003
3     2673
5     2646
2     1546
18    1539
11    1423
6     1175
17     989
12     830
19     825
14     821
1      804
4      755
8      667
10     526
15     489
0      467
Name: cuisine, dtype: int64

In [20]:
#mapping
cuisinemap={'0':'brazilian', '1':'british', '2':'cajun_creole', '3':'chinese', '4':'filipino', '5':'french', '6':'greek', '7':'indian', '8':'irish', '9':'italian', '10':'jamaican', '11':'japanese', '12':'korean', '13':'mexican', '14':'moroccan', '15':'russian', '16':'southern_us', '17':'spanish', '18':'thai', '19':'vietnamese'}

In [21]:
#assigning the Cuisine data as a datframe
Y=[]
Y = df['cuisine']

In [ ]:
#MODELLING -KNN

In [ ]:
#splitting the data in 80:20 ratio for train and test for cusines and ingredients

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100)

In [33]:
#print(X_train)

In [34]:
#print( X_test)

In [46]:
#y_test

In [23]:
KNNMODEL = KNeighborsClassifier(n_neighbors = 12, weights='uniform', algorithm='auto')

In [24]:
KNNMODEL.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=12, p=2,
                     weights='uniform')

In [25]:
#predicting the Cuisines
Cuisinepred = KNNMODEL.predict(X_test)

In [26]:
#accuracy of the model with the predicted and acctual value
accuracy_score(y_test,Cuisinepred)

0.7415461973601508

In [27]:
Cuisinepred

array([ 2, 16,  5, ..., 10, 13,  7])

In [ ]:
#cuisine prediction 

In [28]:
#string of the values which has to be taken
ingredientslist = """romaine lettuce black olives sugar butter eggs"""

In [29]:
sdf=df['jointedingredients']

In [30]:
type(sdf)

pandas.core.series.Series

In [31]:
sDF=sdf.to_frame() 

In [32]:
sDF

,jointedingredients
0,romaine lettuce black olives grape tomatoes ga...
1,plain flour ground pepper salt tomatoes ground...
2,eggs pepper salt mayonaise cooking oil green c...
3,water vegetable oil wheat salt
4,black pepper shallots cornflour cayenne pepper...
...,...
39769,light brown sugar granulated sugar butter warm...
39770,KRAFT Zesty Italian Dressing purple onion broc...
39771,eggs citrus fruit raisins sourdough starter fl...
39772,boneless chicken skinless thigh minced garlic ...


In [34]:
a_row = pd.Series([ingredientslist])

In [35]:
row_df = pd.DataFrame([a_row])

In [36]:
sdf = pd.concat([row_df, sdf], ignore_index=True)

In [37]:
sdf.columns=['jointedingredients']

In [38]:
sdf

,jointedingredients
0,romaine lettuce black olives sugar butter eggs
1,romaine lettuce black olives grape tomatoes ga...
2,plain flour ground pepper salt tomatoes ground...
3,eggs pepper salt mayonaise cooking oil green c...
4,water vegetable oil wheat salt
...,...
39770,light brown sugar granulated sugar butter warm...
39771,KRAFT Zesty Italian Dressing purple onion broc...
39772,eggs citrus fruit raisins sourdough starter fl...
39773,boneless chicken skinless thigh minced garlic ...


In [112]:
type(sdf)

pandas.core.frame.DataFrame

In [ ]:
#vectorizing the jointed data frame

In [39]:
Z = vectorizer.fit_transform(sdf['jointedingredients'])

In [40]:
Z

<39775x3010 sparse matrix of type '<class 'numpy.float64'>'
	with 761958 stored elements in Compressed Sparse Row format>

In [191]:
#print(Z)

In [41]:
#first row - ingredients given
ZERZZ=(Z[0])
    

In [42]:
print(ZERZZ)

  (0, 898)	0.283878100367684
  (0, 375)	0.25713436439867954
  (0, 2647)	0.2294430898733397
  (0, 1868)	0.4445518421131884
  (0, 247)	0.23830410777819644
  (0, 1527)	0.45370569636731634
  (0, 2295)	0.5834269142402961


In [43]:
Z

<39775x3010 sparse matrix of type '<class 'numpy.float64'>'
	with 761958 stored elements in Compressed Sparse Row format>

In [44]:
#remining rows of the ingredients in the data
K=Z[1:,:]

In [45]:
#print(K)

In [46]:
KNNMODEL.fit(K, Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=12, p=2,
                     weights='uniform')

In [47]:
Cui_prediction = KNNMODEL.predict(ZERZZ)

In [48]:
Cui_prediction

array([6])

In [49]:
cv=(Cui_prediction[0])

In [50]:
type(cv)

numpy.int64

In [51]:
a=numpy.array2string(cv)

In [52]:
Cuisine_name=cuisinemap.get(a)


In [ ]:
#the cusine is :

In [53]:
print('Cuisine:',Cuisine_name)

Cuisine: greek


In [54]:
#top 5 closest meals with their distance in parenthesis.

In [ ]:
#calculating cosine distances 

In [97]:
css=(cosine_similarity(ZERZZ,K)).flatten()

In [98]:
type(css)

numpy.ndarray

In [101]:
len(css)

39774

In [102]:
css

array([0.47004517, 0.0964128 , 0.08299575, ..., 0.09276356, 0.0179659 ,
       0.07064212])

In [106]:
dataset = pd.DataFrame({'Cos_sim_distance': css[:, 0]})

IndexError: too many indices for array

In [115]:
g = pd.DataFrame(css)
print(g)


              0
0      0.470045
1      0.096413
2      0.082996
3      0.000000
4      0.052119
...         ...
39769  0.110169
39770  0.102328
39771  0.092764
39772  0.017966
39773  0.070642

[39774 rows x 1 columns]


In [116]:
type(g)

pandas.core.frame.DataFrame

In [117]:
type(df)

pandas.core.frame.DataFrame

In [118]:
g.columns=['cos_sim_distance']

In [119]:
g

,cos_sim_distance
0,0.470045
1,0.096413
2,0.082996
3,0.000000
4,0.052119
...,...
39769,0.110169
39770,0.102328
39771,0.092764
39772,0.017966


In [120]:
h=df['id']

In [124]:
h.columns=['ID']

In [127]:
hh=h.to_frame()

In [128]:
hh

,id
0,10259
1,25693
2,20130
3,22213
4,13162
...,...
39769,29109
39770,11462
39771,2238
39772,41882


In [130]:
finaldf=pd.concat([hh,g], axis=1)

In [133]:
topdf=finaldf.sort_values(by='cos_sim_distance', ascending=False)

In [135]:
print("The top 5 cuisine ids and their distances :")
print(topdf.head(5))

The top 5 cuisine ids and their distances :
          id  cos_sim_distance
34721   3837          0.552896
21028  49064          0.518907
18556  25454          0.504865
20834  18881          0.483116
0      10259          0.470045
